In [61]:
import os
import numpy as np
import tvm
from tvm import te
import vta
from tvm import rpc
from tvm.contrib import util

In [14]:
env = vta.get_env()

In [15]:
# Read Host and Port
host = os.environ.get('VTA_RPC_HOST', '192.168.137.100')
port = int(os.environ.get('VTA_RPC_PORT', '9091'))
host, port

('192.168.137.100', 9091)

In [57]:
if env.TARGET =="pynq":
    # make sure rpc was loaded
    assert tvm.runtime.enabled("rpc") , "Please enable rpc"
    remote = rpc.connect(host, port)
    
    vta.reconfig_runtime(remote)
    vta.program_fpga(remote, bitstream=None)
elif env.TARGET == 'sim':
    remote = rpc.LocalSession()

In [24]:
env.BLOCK_OUT, env.acc_dtype

(16, 'int32')

In [38]:
m = 64
o = 1
A = te.placeholder((o, m, env.BATCH, env.BLOCK_OUT), name='a', dtype=env.acc_dtype)
B = te.placeholder((o, m, env.BATCH, env.BLOCK_OUT), name='b', dtype=env.acc_dtype)

In [41]:
A_buf = te.compute((o, m, env.BATCH, env.BLOCK_OUT), lambda *i: A(*i), name='a_buf')
B_buf = te.compute((o, m, env.BATCH, env.BLOCK_OUT), lambda *i: B(*i), name='b_buf')

In [42]:
f = lambda *i: A_buf(*i).astype(env.acc_dtype) + B_buf(*i).astype(env.acc_dtype)
C_buf = te.compute((o, m, env.BATCH, env.BLOCK_OUT), f, name='c_buff')

In [43]:
C = te.compute((o, m, env.BATCH, env.BLOCK_OUT), lambda *i: C_buf(*i).astype(env.inp_dtype), name='c')

In [44]:
s = te.create_schedule(C.op)

In [45]:
s[A_buf].set_scope(env.acc_scope)
s[B_buf].set_scope(env.acc_scope)
s[C_buf].set_scope(env.acc_scope)

stage(c_buff, 0x7fffbbaeef80)

In [48]:
s[A_buf].op.axis[0]

iter_var(i0, range(min=0, ext=1))

In [49]:
s[A_buf].pragma(s[A_buf].op.axis[0], env.dma_copy)
s[B_buf].pragma(s[B_buf].op.axis[0], env.dma_copy)
s[C].pragma(s[C].op.axis[0], env.dma_copy)

In [50]:
# Tell TVM that the computation needs to be performed
# on VTA's vector ALU
s[C_buf].pragma(C_buf.op.axis[0], env.alu)

# Let's take a look at the finalized schedule
print(vta.lower(s, [A, B, C], simple_mode=True))

primfn(a_1: handle, b_1: handle, c_1: handle) -> ()
  attr = {"global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(int32), int32, [1, 64, 1, 16], []),
             c: Buffer(c_2: Pointer(int8), int8, [1, 64, 1, 16], []),
             b: Buffer(b_2: Pointer(int32), int32, [1, 64, 1, 16], [])}
  buffer_map = {a_1: a, b_1: b, c_1: c} {
  attr [a_buf: Pointer(int32)] "storage_scope" = "local.acc_buffer" {
    attr [IterVar(vta: int32, (nullptr), "ThreadIndex", "vta")] "coproc_scope" = 2 {
      @tir.call_extern("VTALoadBuffer2D", @tir.tvm_thread_context(@tir.vta.command_handle(, dtype=handle), dtype=handle), a_2, 0, 64, 1, 64, 0, 0, 0, 0, 0, 3, dtype=int32)
      @tir.call_extern("VTALoadBuffer2D", @tir.tvm_thread_context(@tir.vta.command_handle(, dtype=handle), dtype=handle), b_2, 0, 64, 1, 64, 0, 0, 0, 0, 64, 3, dtype=int32)
      attr [IterVar(vta, (nullptr), "ThreadIndex", "vta")] "coproc_uop_scope" = "VTAPushALUOp" {
        @tir.call_extern("VTAUopLoopBeg

In [51]:
my_vadd = vta.build(s, [A, B, C], "ext_dev", env.target_host, name="my_vadd")

In [59]:
# Write the compiled module into an object file.
temp = util.tempdir()
my_vadd.save(temp.relpath("vadd.o"))

# Send the executable over RPC
remote.upload(temp.relpath("vadd.o"))
f = remote.load_module("vadd.o")

In [64]:
# Get the remote device context
ctx = remote.ext_dev(0)

# Initialize the A and B arrays randomly in the int range of (-128, 128]
A_orig = np.random.randint(-128, 128, size=(o * env.BATCH, m * env.BLOCK_OUT)).astype(A.dtype)
B_orig = np.random.randint(-128, 128, size=(o * env.BATCH, m * env.BLOCK_OUT)).astype(B.dtype)

# Apply packing to the A and B arrays from a 2D to a 4D packed layout
A_packed = A_orig.reshape(o, env.BATCH, m, env.BLOCK_OUT).transpose((0, 2, 1, 3))
B_packed = B_orig.reshape(o, env.BATCH, m, env.BLOCK_OUT).transpose((0, 2, 1, 3))

# Format the input/output arrays with tvm.nd.array to the DLPack standard
A_nd = tvm.nd.array(A_packed, ctx)
B_nd = tvm.nd.array(B_packed, ctx)
C_nd = tvm.nd.array(np.zeros((o, m, env.BATCH, env.BLOCK_OUT)).astype(C.dtype), ctx)

# Invoke the module to perform the computation
f(A_nd, B_nd, C_nd)

RPCError: Traceback (most recent call last):
  [bt] (8) /home/lei/tvm/build/libtvm.so(TVMArrayAlloc+0xd6) [0x7f4fe988c0d6]
  [bt] (7) /home/lei/tvm/build/libtvm.so(tvm::runtime::NDArray::Empty(std::vector<long, std::allocator<long> >, DLDataType, DLContext)+0x20a) [0x7f4fe988bf5a]
  [bt] (6) /home/lei/tvm/build/libtvm.so(tvm::runtime::RPCDeviceAPI::AllocDataSpace(DLContext, unsigned long, unsigned long, DLDataType)+0x167) [0x7f4fe98ba9d7]
  [bt] (5) /home/lei/tvm/build/libtvm.so(non-virtual thunk to tvm::runtime::RPCClientSession::AllocDataSpace(DLContext, unsigned long, unsigned long, DLDataType)+0x222) [0x7f4fe98c3602]
  [bt] (4) /home/lei/tvm/build/libtvm.so(+0x128793f) [0x7f4fe98bd93f]
  [bt] (3) /home/lei/tvm/build/libtvm.so(tvm::runtime::RPCEndpoint::HandleUntilReturnEvent(bool, std::function<void (tvm::runtime::TVMArgs)>)+0x23e) [0x7f4fe98bd56e]
  [bt] (2) /home/lei/tvm/build/libtvm.so(tvm::runtime::RPCEndpoint::EventHandler::HandleNextEvent(bool, bool, std::function<void (tvm::